Python script by [__Hassan Mojeed__](https://hassanmojeed.pages.dev)<br>
Email: mojeed.o.hassan@gmail.com<br>
Website: [https://hassanmojeed.pages.dev](https://hassanmojeed.pages.dev)


## Introduction

This project intends to use python script to device an ETL (Extract, Transform and Load) Process. The script will fetch excels files from the designated directory and conduct some data transformation before loading to Google BigQuery.

## Extraction Phase

We have over 50 excel file formats with each file having exact same data format with the others. Creating a **For Loop** will automate the data extraction.

## Transfromation Phase

Uniformity and standardization is key in any data analysis hence, carefully studying the data shape and data types to ensure that all data in each field follow the same format are executed in this phase of the ETL process.

## Loading Phase

The destination data warehouse is the Google BigQuery where further anylysis and insight can be executed. With the help of the Pandas-gbq package this stage was achieved seamlessly.

## Check out the automated Dashbaord [here](https://app.powerbi.com/view?r=eyJrIjoiMGFmYmI4ZmEtYTM3ZS00NzM1LTlhNjItOGNlMGEyNjk4Y2M5IiwidCI6ImFlM2E5OTA2LTc4MWEtNDQ2YS1iZGI2LTYzNzdjMDllMmM2ZiIsImMiOjF9)

In [1]:
import pandas as pd
import numpy as np
import os
from glob import glob
import warnings
import pandas_gbq
from io import StringIO

# Ignore warnings to maintain clean output
warnings.filterwarnings('ignore')

## Extraction Phase

In [2]:
# Get the current working directory and set the data folder path
pwd = os.getcwd() + '/data folder'

# Find all Excel files in the data folder
excel_files_path = glob(pwd + '/*.xlsx')
excel_files_path

['/Users/mj/Projects/Projects/Advertising/data folder/Dec-2016.xlsx',
 '/Users/mj/Projects/Projects/Advertising/data folder/Jun-2016.xlsx',
 '/Users/mj/Projects/Projects/Advertising/data folder/Nov-2016.xlsx',
 '/Users/mj/Projects/Projects/Advertising/data folder/May-2016.xlsx',
 '/Users/mj/Projects/Projects/Advertising/data folder/Mar-2017.xlsx',
 '/Users/mj/Projects/Projects/Advertising/data folder/Mar-2016.xlsx',
 '/Users/mj/Projects/Projects/Advertising/data folder/May-2017.xlsx',
 '/Users/mj/Projects/Projects/Advertising/data folder/Nov-2017.xlsx',
 '/Users/mj/Projects/Projects/Advertising/data folder/Jun-2017.xlsx',
 '/Users/mj/Projects/Projects/Advertising/data folder/Dec-2017.xlsx',
 '/Users/mj/Projects/Projects/Advertising/data folder/Jul-2018.xlsx',
 '/Users/mj/Projects/Projects/Advertising/data folder/Oct-2018.xlsx',
 '/Users/mj/Projects/Projects/Advertising/data folder/Sep-2018.xlsx',
 '/Users/mj/Projects/Projects/Advertising/data folder/Apr-2016.xlsx',
 '/Users/mj/Projects

In [3]:
# Create an empty list to store dataframes
data = []

# Iterate over each Excel file path
for file in excel_files_path:
    # Read the Excel file into a DataFrame
    df = pd.read_excel(file)
    # Append the DataFrame to the list
    data.append(df)

# Concatenate all DataFrames in the list into one DataFrame
data_frame = pd.concat(data)

In [4]:
# Filter out rows where TotalSpend is zero
df = data_frame[data_frame['TotalSpend'] != 0]

In [5]:
# Create a copy of the DataFrame for advertisement data
ad_data = df.copy()

# Display the shape of the advertisement data
ad_data.shape

(628961, 13)

In [6]:
# Display information about the advertisement data
ad_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 628961 entries, 0 to 11616
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Year          628961 non-null  int64  
 1   Month         628961 non-null  object 
 2   Quarter       628961 non-null  object 
 3   Media         628961 non-null  object 
 4   Zone          628940 non-null  object 
 5   Category      628961 non-null  object 
 6   Advertizer    628961 non-null  object 
 7   Brand         628956 non-null  object 
 8   Main Station  628959 non-null  object 
 9   Station       628959 non-null  object 
 10  State         628940 non-null  object 
 11  TotalSpend    628961 non-null  float64
 12  TotalSpots    628961 non-null  int64  
dtypes: float64(1), int64(2), object(10)
memory usage: 67.2+ MB


## Transformation Phase

In [7]:
# Extract the month from the 'Month' column and replace abbreviations with full month names
ad_data["Month"] = ad_data['Month'].str.split("-", expand=True)[0].replace({
                                            "Jan": "January",
                                            "Feb": "February",
                                            "Mar": "March",
                                            "Apr": "April",
                                            "May": "May",
                                            "Jun": "June",
                                            "Jul": "July",
                                            "Aug": "August",
                                            "Sep": "September",
                                            "Oct": "October",
                                            "Nov": "November",
                                            "Dec": "December"
})



# Capitalize the first letter of specified columns

columns =['Media','Zone', 'Category', 'Advertizer',
       'Brand', 'Main Station', 'Station', 'State']

for column in columns:
    ad_data[f'{column}'] = ad_data[f'{column}'].str.title().str.strip()


# Calculate the cost per slot and add it as a new column
    
ad_data['CostPerSlot'] = ad_data['TotalSpend']/ad_data['TotalSpots']

# Applying the operation
ad_data['YearMonth'] = ad_data.apply(lambda row: str(row['Year']) + "-" + row['Month'], axis=1)

# Convert to datetime
ad_data['YearMonth'] = pd.to_datetime(ad_data['YearMonth']).dt.strftime('%Y-%m')

ad_data['MonthNumber'] = pd.to_datetime(ad_data['YearMonth']).dt.month

ad_data['QuarterNumber'] =pd.to_datetime(ad_data['YearMonth']).dt.quarter


# Replace values in 'Main Station' and 'Station' columns with 'Out of Home' and 'Press' if they are 'No Station'

choose_columns =['Main Station', 'Station']

for column in choose_columns:
    
    ad_data[f'{column}'] = ad_data.apply(lambda row: 'Out of Home'
                                        if row[f'{column}'] == 'No Station'
                                            and row['Media'] == 'Outdoor'
                                        else('Press' if row[f'{column}'] == 'No Station'
                                             and row['Media'] == 'Press'
                                        else row[f'{column}']), axis =1)
    
ad_data['Media'] = ad_data.apply(lambda row: 'Cable'
                                 if row['Station'] in ['Channels Dstv', 'Ontv Max Dstv']
                                 else row['Media'], axis = 1)

In [8]:
# Map certain brand names to standard ones using a predefined dictionary

brand_mapping = {

    'Amel Susan':'Amel Susan Cocoa Powder',
    'Access Bank Nig. Plc.': 'Access Bank',
    'Always Ultra': 'Always',
    'Always Range': 'Always',
    'Always Safe': 'Always',
    'Amel Susan': 'Amel Susan Cocoa Powder',
    'Amphora': 'Amphora Wine',
    'Ariel Advance Formula': 'Ariel',
    'Ariel Enzymax': 'Ariel',
    'Ariel Prozim': 'Ariel',
    'Ariel Fresh': 'Ariel',
    'Ariel Automatic': 'Ariel',
    'Audi A4': 'Audi',
    'Audi Motors': 'Audi',
    'Audi A6': 'Audi',
    'Audi 5': 'Audi',
    'Audi Range': 'Audi',
    'Bbc World Service' : 'Bbc',
    'Baby & Mama Bodyy Cream' : 'Baby & Mama Body Cream',
    'Baby & Mama Bodyy Cream': 'Baby & Mama Body Cream',
    'Beauty Peagant&Contest': 'Beauty Peagant/Contest',
    'Beloxxi Crackers': 'Beloxxi Biscuits',
    'Binta In\'T School': 'Binta Int L School',
    'Bull Dark Rum': 'Bull Dark Rum & Gin',
    'Bonababe' : 'Bonababe Blood Tonic',
    'Briscoe Ford' : 'Briscoe Motor',
    'Bristow' : 'Bristow Airways',
    'Bull Dark Rum' : 'Bull Dark Rum & Gin',
    'Century' : 'Century Appliances',
    'C Way Fruit Juice': 'CWay Fruit Juice',
    'C Way Water Dispenser':'CWay Water',
    'CWay Water Dispenser':'CWay Water',
    'C Way Water': 'CWay Water',
    'Chapeau' : 'Chapeau Wine',
    'Cheveron Nig Ltd (Corp)' : 'Chevron Nig Ltd (Corp)',
    'Chi Ice Tea/Hollandia Evap': 'Chi Ice Tea',
    'Chi Exotic/Chi Ice Tea': 'Chi Ice Tea',
    'Chi Exotic Banana': 'Chi Exotic',
    'Chi-Exotic/Hollandia Milk 215Gm': 'Chi Exotic',
    'Chi Health Soya Milk': 'Chi Milk',
    'Chikki': 'Chikki Noodles',
    'Chivas' : 'Chivas Regal',
    'Close-Up Eros': 'Close Up Toothpaste',
    'Close Up Deep Cleaning': 'Close Up Toothpaste',
    'Close Up Red Hot': 'Close Up Toothpaste',
    'Close Up Fire Freeze': 'Close Up Toothpaste',
    'Close Up Herbal': 'Close Up Toothpaste',
    'Close-Up Red Gel': 'Close Up Toothpaste',
    'Close Up Deep Action': 'Close Up Toothpaste',
    'Close Up Cool Breeze': 'Close Up Toothpaste',
    'Coast Evaporated Milk': 'Coast Evap & Powdered Milk',
    'Cow Bell Vitarich': 'Cowbell Milk',
    'Cowbell Powdered Milk': 'Cowbell Milk',
    'Cowbell Evap Milk': 'Cowbell Milk',
    'Cowbell Sweet Milk': 'Cowbell Milk',
    'Cusson\'S Baby Range': 'Cussons Baby Range',
    'Dabur Herbal Green Gel': 'Dabur',
    'Dabur Med Natural Tpaste': 'Dabur',
    'Dabur Herbal': 'Dabur',
    'Dark Sailor Rum , Seaman Schnapps Rum, Calypso': 'Dark Sailor Rum',
    'Dettol Soap Original': 'Dettol Soap',
    'Dettol Cool': 'Dettol Soap',
    'Dettol Fresh': 'Dettol Soap',
    'Dettol Herbal Soap': 'Dettol Soap',
    'Dettol Skin Care Soap': 'Dettol Soap',
    'Dettol Range Cleansing': 'Dettol Soap',
    'Emzor Pharmaceuticals Nig.Ltd (Corp' : 'Emzor Pharmaceuticals (Corp)',
    'Etisalat (Emts)': 'Etisalat/9Mobile',
    'Etisalat(Emts)': 'Etisalat/9Mobile',
    '9Mobile': 'Etisalat/9Mobile',
    'Fanta Orange': 'Fanta Range',
    'Fanta Apple': 'Fanta Range',
    'Fanta Pineapple': 'Fanta Range',
    'Fanta Chapman': 'Fanta Range',
    'Fanta Tonic': 'Fanta Range',
    'Fbn Life Insurance': 'Fbn Insurance',
    'Gillette Mach3': 'Gillette',
    'Gillette Blue 3': 'Gillette',
    'Gillette Sensor Razor': 'Gillette',
    'Gillette Blue 11': 'Gillette',
    'Gino Spices': 'Gino',
    'Gino Chicken Cubes': 'Gino',
    'Gino Cubes': 'Gino',
    'G/Penny Flour': 'Golden Penny Flour',
    'Go Tv' : 'Gotv',
    'Guinea Insurance Plc (Corp)': 'Guinea Insurance Plc',
    'Harpic Hygiene' : 'Harpic',
    'Harpic Power Plus' : 'Harpic',
    'Hollandia Evaporated Milk 125Gms': 'Hollandia Evaporated Milk',
    'Hollandia Uht Milk /Chi Happy Hour': 'Hollandia Evaporated Milk',
    'Hollandia Malt & Milk': 'Hollandia Yoghurt',
    'Hollandia Yoghurt(1 Liter)': 'Hollandia Yoghurt',
    'Hollandia Evaporated Milk/Custard': 'Hollandia Yoghurt',
    'Hollandia Flavoured Milk': 'Hollandia Yoghurt',
    'Honda Accord': 'Honda Vehicles',
    'Honda Cr-V': 'Honda Vehicles',
    'Honewell Engine Oil': 'Honeywell Engine Oil',
    'Hyundai Range': 'Hyundai Vehicles',
    'Hyundai Elentra': 'Hyundai Vehicles',
    'Hyundai Accent': 'Hyundai Vehicles',
    'Hyundai Tucson': 'Hyundai Vehicles',
    'Hyundai Truck': 'Hyundai Vehicles',
    'Innoson Motors (Ivm)': 'Innoson Vehicles',
    'Innoson Vehincles': 'Innoson Vehicles',
    'Kia Sportage': 'Kia Vehicles',
    'Kia Carens': 'Kia Vehicles',
    'Kia Range': 'Kia Vehicles',
    'Kia Cerato': 'Kia Vehicles',
    'Kia Rio': 'Kia Vehicles',
    'Kia Optima': 'Kia Vehicles',
    'Knorr Cube': 'Knorr Seasoning',
    'Knorr Chicken': 'Knorr Seasoning',
    'Knorr Seasoning Mix': 'Knorr Seasoning',
    'Knorr Naija Pot': 'Knorr Seasoning',
    'Knorr Signature': 'Knorr Seasoning',
    'Lord\'S Dry Gin': 'Lords Dry Gin',
    'Lambweston Poundo Potato Flakes' : 'Lambweston Poundo Potato',
    'Lumapli': 'Lumapil',
    'Lux': 'Lux Range',
    'Lokmal' : 'Lokmal Malaria Drugs',
    'Lucozade Sport' : 'Lucozade Boost',
    'Leadway Assurance Co.Ltd.' : 'Leadway Assurance',
    'Macho Man': 'Machoman Energy Drink',
    'Macleans Complete Care': 'Macleans',
    'Macleans Milk Teeth': 'Macleans',
    'Macleans Freshmint': 'Macleans',
    'Maggi Chicken': 'Maggi Seasonings',
    'Maggi Cube': 'Maggi Seasonings',
    'Maggi Mix Powder': 'Maggi Seasonings',
    'Maggi Naija Pot': 'Maggi Seasonings',
    'Maggi Signature': 'Maggi Seasonings',
    'Mahindra Arrow': 'Mahindra',
    'Mamadol Vegetable Oil' : 'Mamador Oil',
    'Mirinda Orang': 'Mirinda',
    'Mirinda Red Apple': 'Mirinda',
    'Mirinda Apple': 'Mirinda',
    'Mirinda Range': 'Mirinda',
    'Mobil Oil Plc (Corp)': 'Mobil Oil Nig. Plc (Corp)',
    'Mobil Oil Nig.Plc (Corp)': 'Mobil Oil Nig. Plc (Corp)',
    'Mouka Foam(New Flora)': 'Mouka Foam',
    'Mouka Royal Foam': 'Mouka Foam',
    'Mr Chef Seasoning Cube': 'Mr Chef Seasoning',
    'Mr Chef Seasoning Powder': 'Mr Chef Seasoning',
    'Mr Cheff Iodine Salt': 'Mr Chef Seasoning',
    'Nescafe Breakfast': 'Nescafe',
    'Nescafe Classic': 'Nescafe',
    'Nissan Oval': 'Nissan',
    'Nissan Qashdai': 'Nissan',
    'Nissan Tida': 'Nissan',
    'Nissan Range': 'Nissan',
    'Omega 3': 'Omega',
    'Omega H3': 'Omega',
    'Onga Cubes': 'Onga Seasoning',
    'Onga Range': 'Onga Seasoning',
    'Onga Seasoning Powder': 'Onga Seasoning',
    'Paediatric Ass Of Nig': 'Paediatric Association Of Nigeria',
    'Path Finder Hotel' : 'PathFinder Hotel',
    'Pampers Active Baby': 'Pampers',
    'Panadol Children': 'Panadol',
    'Panadol Extra': 'Panadol',
    'Panadol Optizorb': 'Panadol',
    'Peak Evaporated In Sachet': 'Peak Milk',
    'Peak Evaporated Milk': 'Peak Milk',
    'Pepsi Light': 'Pepsi',
    'Peugeot 206': 'Peugeot',
    'Premier Cool Dro Soap': 'Premier Cool Deo Soap',
    'Prag' : 'Prag Solar',
    'Rambo Insecticide (Magic)' : 'Rambo Insecticide',
    'Rima Radio Sokoto': 'Rima F.M Sokoto',
    'Riunite Lambrusco': 'Riunite Lambrusco Wine',
    'Rose Daniel Wine': 'Rose Daniel Wine & J&D Fruit Drink',
    'Royal Eagle Premium Cream Liqueur': 'Royal Eagle',
    'Savlon' : 'Savlon Antiseptic Liquid',
    'Shell Petroleum Dev.Coy (Corp)' : 'Shell Petroleum (Corp)',
    'Sigma Pension': 'Sigma Pensions',
    'Silverbird Communications': 'Silverbird Televission (Stv)',
    'Slot Phone' : 'Slot',
    'Star' : 'Star Beer',
    'Sudrex Paracetamol' : 'Sudrex',
    'Sumeta Antimalaria' : 'Sumether',
    'Supreme Flavour Soya Milk': 'Supreme Flavour Milk',
    'Tcb Hair Relaxer' : 'Tcb Hair Care',
    'Terra Seasoning Cube': 'Terra Seasonings',
    'Terra Spice': 'Terra Seasonings',
    'Tom Tom Extra': 'Tom Tom',
    'Tom-Tom': 'Tom Tom',
    'Tom Tom Honey Lemon': 'Tom Tom',
    'Tom Tom Range': 'Tom Tom',
    'Toyota Land Cruiser': 'Toyota',
    'Transcorp': 'Transcorp Hotels',
    'Tura': 'Tura Antiseptic Soap',
    'Tvc Programme' : 'Tv Continental',
    'U.B.A.': 'U.B.A',
    'U.B.A.Plc (Corp)': 'U.B.A',
    'Union Bank (Corp)': 'Union Bank Of Nig (Corp)',
    'Unity Bank': 'Unity Bank Plc.',
    'Viju Drinking Yoghurt' : 'Viju Milk',
    'Vital Feeds': 'Vital Fish Feed',
    'Vitafish Feed': 'Vital Fish Feed',
    'Wazobia Fm': 'Wazobia',
    'Zedd Moblie': 'Zen Mobile',
    'Zedd Mobile': 'Zen Mobile'
}

# Replace the brand names using the mapping dictionary
ad_data['Brand'] = ad_data['Brand'].replace(brand_mapping) 

ad_data['Brand'] = ad_data.apply(lambda row: 'Total Engine Oil'
                                 if row['Category'] == 'Lubricants'
                                 and row['Brand'] == 'Total'
                                 else row['Brand'], axis = 1
)

In [9]:
# Apply a lambda function to each row of the DataFrame to assign 'Advertizer' values based on 'Brand' values
ad_data['Advertizer'] = ad_data.apply(lambda row: 
                                      'Cway' if row['Brand'] in ['Cway Iced Tea', 'Cway Fruit Juice', 'Cway Water']
                                      else 'Ashaka Cement' if row['Brand'] in ['Ashaka Cement Coy (Corp)', 'Ashaka Cement']
                                      else 'Aiico Ins. Plc' if row['Brand'] in ['Aiico Insurance', 'Aiico Pension Mangers']
                                      else 'Amel Internatnal Services Ltd' if row['Brand'] == 'Amel Susan Cocoa Powder'
                                      else 'Arm Company Ltd' if row['Brand'] == 'Arm Pension Management'
                                      else 'Aromes 3 Lions' if row['Brand'] in ['Aromes 3 Lions','Aromes 3 Lions Exotic Flavour']
                                      else 'Bristish Broadcasting Coorperation' if row['Brand'] == 'Bbc World Service'
                                      else 'Beloxxi Industries Ltd' if row['Brand'] == 'Beloxxi Biscuits'
                                      else 'Bristow Group' if row['Brand'] == 'Bristow Airways'
                                      else 'Tea & Coffee' if row['Brand'] == 'Chi Ice Tea'
                                      else 'Chivas Brothers Ltd' if row['Brand'] == 'Chivas Regal'
                                      else 'Erisco Foods Ltd' if row['Brand'] == 'Erisco Foods Limited'
                                      else 'Gino Foods Africa Holdco' if row['Brand'] in ['Gino Tomatoes', 'Gino Mackerel']
                                      else 'Nutricima' if row['Brand'] == 'Nutri C Milk'
                                      else 'Prag Global' if row['Brand'] == 'Prag Solar'
                                      else 'Briscoe' if row['Brand'] == 'Briscoe Motor'
                                      else 'Johnson & Johnson' if row['Brand'] == 'Savlon Antiseptic Liquid'
                                      else 'Orange Drugs Limited' if row['Brand'] == 'Sudrex'
                                      else 'Tasty Time Nig. Ltd' if row['Brand'] in ['Tasty Time', 'Tasty Time Energy Drink']
                                      else 'Tcb' if row['Brand'] == 'Tcb Hair Care'
                                      else 'Grand Cereal Oil Mill Nig.Ltd.' if row['Brand'] == 'Vital Fish Feed'
                                      else 'Zaron' if row['Brand'] == 'Zaron Costmetics'
                                      else row['Advertizer'], axis=1)


# Define replacements for certain advertisers and apply them to the 'Advertiser' column
replacements = {
    'Chapeau Wine' : 'Intercontinental Distillers Lt',
    'Etisalat (Emts)': 'Etisalat/9Mobile',
    '9Mobile': 'Etisalat/9Mobile',
    'Diamond Bank Plc' : 'Diamond/Access Bank Nig Plc',
    'Access Bank Nig Plc' : 'Diamond/Access Bank Nig Plc',
    'Sonik Blenders': 'Sonik Electronics',
    'Divine Source Ventures': 'Divine Source Int Ventures',
    'Honey Well Nig Plc': 'Honeywell Group',
    'Htc': 'Htc Nigeria',
    'Honeywell Engine Oil': 'Honeywell Oil & Gas Ltd',
    'Innoson Vehincles': 'Innoson Vehicle Man. Co. Ltd',
    'Kold Time Inhaler': 'Embassy Pharm & Chem Ltd',
    'Lokmal Malaria Drugs': 'Emzor Pharmaceuticals Ltd.',
    'Lumapil': 'Phamatex Industries Ltd',
    'Mahindra': 'Mahindra Group',
    'Paediatric Association Of Nigeria': 'Boulous',
    'Panasonic': 'Panabiz Industries Limited',
    'Rose Daniel Wine & J&D Fruit Drink': 'Fusedam Nig Ltd',
    'Sigma Pensions': 'Sigma Pensions',
    'Transcorp Hotels': 'Transcorp Hotels',
    'U.B.A. Moneygram': 'U.B.A',
    'Viju Milk': 'Viju Industries Ltd',
    'Viju Fruit Juice': 'Viju Industries Ltd',
    'Viju Water': 'Viju Industries Ltd',
    'Zen Mobile': 'Zen',

    'Other Cosmetic Range Ads': lambda row: row['Brand'],
    'Sundary Ad (Wine & Liqueur)': lambda row: row['Brand'],
    'Sundary Waters': lambda row: row['Brand'],
    'Sundry (Household Goods)': lambda row: row['Brand'],
    'Sundry Ad (Building Materials)': lambda row: row['Brand'],
    'Sundry Ad (Other Inform.Servic': lambda row: row['Brand'],
    'Sundry Ad (Stationery & Books)': lambda row: row['Brand'],
    'Sundry Ad.Capital Goods&Equip.': lambda row: row['Brand'],
    'Sundry Advertisers (Clothings)': lambda row: row['Brand'],
    'Sundry Pharmaceutical': lambda row: row['Brand'],
    'Sundry Advertisrs (Drinks)': lambda row: row['Brand'],
    'Sundry Advertisiers (Foods)': lambda row: row['Brand'],
    'Sundry Advertisiers(Services)': lambda row: row['Brand'],
    'Sundry Advertisers(Petroleum)': lambda row: row['Brand'],
    'Sundry Advertisers(Chemicals)': lambda row: row['Brand'],
    'Sundry Advertisers(Confectiona': lambda row: row['Brand'],
    'Sundry Agriculture': lambda row: row['Brand'],
    'Sundry Advertisers (Cosmetics)': lambda row: row['Brand'],
    'Sundry Advertisier[Foods]': lambda row: row['Brand'],
    'Sundry Advertisers(Soft Drink)': lambda row: row['Brand'],
    'Sundry Nutritional Drink': lambda row: row['Brand'],
    'Sundry Foods Restaurant': lambda row: row['Brand']
}

for key, value in replacements.items():
    if callable(value):
        ad_data['Advertizer'] = ad_data.apply(value, axis=1)
    else:
        ad_data.loc[ad_data['Advertizer'] == key, 'Advertizer'] = value



In [10]:


# First transformation: assigning 'Category' based on 'Brand'
ad_data['Category'] = ad_data.apply(lambda row: 
                                    'Tea & Coffee' if row['Brand'] == 'Chi Ice-Tea'
                                    else 'Shopping' if row['Brand'] == 'Slot'
                                    else row['Category'], axis=1)

# Second transformation: adjusting 'Category' for specific conditions
ad_data['Category'] = ad_data.apply(lambda row: 'Building Materials' 
                                    if (row['Category'] == 'Others' 
                                        and row['Advertizer'] == 'Sundry Ad (Building Materials)') 
                                        or row['Category'] == 'Others Building Materials'
                                    else row['Category'], axis=1)


# Replace categories in the 'Category' column with standardized ones
def replace_categories(ad_data):
    replacements = {
        'Multi-Product' : 'Multi-Products',
        'Multi-Product(Foods)' : 'Multi-Products',
        'Rel002': 'Religion',
        'Ble001': 'Blenders',
        'Rel002': 'Religion',
        'Durable': 'Durables',
        'Fan001': 'Fans',
        'Nutritional Drinks': 'Nutritional Drinks Rtd',
        'Online Mall': 'Online Business',
        'Others Clothings': 'Clothings',
        'Others Foods': 'Foods',
        'Others Other Infor.Announc': 'Announcements',
        'Others Pharmaceutical': 'Pharmaceutical',
        'Others Services': 'Services',
        'Yoghurt Rtd': 'Yoghurt & Rtd'
    }

    ad_data['Category'] = ad_data['Category'].replace(replacements)

replace_categories(ad_data)



# Display the first few rows of the processed advertisement data

ad_data.head()

,Year,Month,Quarter,Media,Zone,Category,Advertizer,Brand,Main Station,Station,State,TotalSpend,TotalSpots,CostPerSlot,YearMonth,MonthNumber,QuarterNumber
0,2016,December,Quarter 4,Outdoor,South South,Air Conditioners,Samsung Aircondition,Samsung Aircondition,Out of Home,Out of Home,Delta State,60000.00,1,60000.00,2016-12,12,4
1,2016,December,Quarter 4,Outdoor,South West,Analgesics,Emzor Paracetamol,Emzor Paracetamol,Out of Home,Out of Home,Ekiti State,250000.00,1,250000.00,2016-12,12,4
2,2016,December,Quarter 4,Outdoor,North Central,Analgesics,Cannon Extra,Cannon Extra,Out of Home,Out of Home,Niger State,33333.33,1,33333.33,2016-12,12,4
3,2016,December,Quarter 4,Outdoor,North Central,Analgesics,Sudrex,Sudrex,Out of Home,Out of Home,Kwara State,33333.33,1,33333.33,2016-12,12,4
4,2016,December,Quarter 4,Outdoor,North West,Analgesics,Sudrex,Sudrex,Out of Home,Out of Home,Zamfara State,33333.33,1,33333.33,2016-12,12,4


In [11]:
ad_data.columns = ad_data.columns.str.replace(" ","")

# List of column names to be converted to string type
columns_str = ['Year', 'Month', 'Quarter', 'Media', 'Zone', 'Category', 'Advertizer', 'Brand', 'MainStation', 'Station', 'State']

# Convert specified columns to string type
for column in columns_str:
    ad_data[column] = ad_data[column].astype(str)

# Convert specified numerical columns to float type
columns_int = ['TotalSpend', 'TotalSpots', 'CostPerSlot']

for column in columns_int:
    ad_data[column] = ad_data[column].astype(int)

## Loading Phase

In [12]:
#Writing the dataframe to a BQ table

table = "Industry.advertising data"
project_id = "cool-ship-407420"

pandas_gbq.to_gbq(ad_data,destination_table=table, project_id=project_id, if_exists="replace")